In [2]:
import os
from dataclasses import dataclass
import logging
logging.basicConfig(level=logging.INFO, 
    format='%(asctime)s %(levelname)s - %(message)s',
    datefmt="%H:%M:%S"
)

import numpy as np
%rm -rf my-experiment*
%rm -rf ioh_data*

In [3]:
import ioh
%pip show ioh

Name: ioh
Version: 0.3.3
Summary: The experimenter for Iterative Optimization Heuristics
Home-page: None
Author: Jacob de Nobel, Furong Ye, Diederick Vermetten, Hao Wang, Carola Doerr and Thomas Bäck
Author-email: iohprofiler@liacs.leidenuniv.nl
License: None
Location: /home/jacob/code/IOHexperimenter
Requires: numpy
Required-by: 
Note: you may need to restart the kernel to use updated packages.


# Problem
In ioh, everything revolves around the `problem` class, which exists for both `Real` and `Integer` types for continious and discrete problems resp. These classes are wrappers around an objective function, and can be used to interact with the various other parts of iohexperimenter. 

We have a number of objective functions already implemented for convenience, which includes the objective functions from the BBOB single objective benchmark by the COCO platform for the continous case and the PBO benchmark functions for the discrete case.


In [4]:
# A list of problems can be accessed via the base classes
ioh.problem.Real.problems

{1: 'Sphere',
 2: 'Ellipsoid',
 3: 'Rastrigin',
 4: 'BuecheRastrigin',
 5: 'LinearSlope',
 6: 'AttractiveSector',
 7: 'StepEllipsoid',
 8: 'Rosenbrock',
 9: 'RosenbrockRotated',
 10: 'EllipsoidRotated',
 11: 'Discus',
 12: 'BentCigar',
 13: 'SharpRidge',
 14: 'DifferentPowers',
 15: 'RastriginRotated',
 16: 'Weierstrass',
 17: 'Schaffers10',
 18: 'Schaffers1000',
 19: 'GriewankRosenBrock',
 20: 'Schwefel',
 21: 'Gallagher101',
 22: 'Gallagher21',
 23: 'Katsuura',
 24: 'LunacekBiRastrigin'}

In [5]:
# In order to instantiate a problem instance, we can do the following:
problem = ioh.get_problem(
    "Sphere", 
    instance=1,
    dimension=10,
    problem_type="Real"
)
problem

<RealProblem 1. Sphere (iid=1 dim=10)>

In [6]:
# The problem class includes information about the problem, which can be retrieved via the meta_data accessor
problem.meta_data.name

'Sphere'

In [7]:
# The current state of the problem, e.g. the number of evaluations, best seen points etc. are stored in the problems state.
problem.state

<State evaluations: 0 optimum_found: false current_best: <Solution x: {nan, nan, nan, nan, nan, nan, nan, nan, nan, nan} y: nan>>

In [8]:
# Every problem as has a simple box-bounds associcated
problem.bounds

<BoxConstraint lb: [{-5, -5, -5, -5, -5, -5, -5, -5, -5, -5}] ub: [{5, 5, 5, 5, 5, 5, 5, 5, 5, 5}]>

In [9]:
# We can access the contraint information of the problem
x0 = np.random.uniform(problem.bounds.lb, problem.bounds.ub)

# Evaluation happens like a 'normal' objective function would
problem(x0)

# Whenever the problem is evaluated, the state changes
problem.state

<State evaluations: 1 optimum_found: false current_best: <Solution x: {-3.00524507124168, 3.9376814601934047, 1.1233170153846759, 1.7038057367456672, -4.103640188634418, 1.6097893891088644, 0.43541476537949286, -1.4898899565233847, -3.3593049944783617, -4.928526573882604} y: 132.29079108838306>>

In [10]:
# If we want to perform multiple runs with the same objective function, after every run, the problem has to be reset, 
# such that the internal state reflects the current run.
def run_experiment(problem, algorithm, n_runs=5):
    for run in range(n_runs):
        
        # Run the algorithm on the problem
        algorithm(problem)

        # print the best found for this run
        print(f"run: {run+1} - best found:{problem.state.current_best.y: .3f}")

        # Reset the problem
        problem.reset()

In [11]:
@dataclass
class RandomSearch:
    'Simple random search algorithm'
    
    n: int
    length: float = 0.0
        
    def __call__(self, problem: ioh.problem.Real) -> None:
        'Evaluate the problem n times with a randomly generated solution'
        
        for _ in range(self.n):
            # We can use the problems bounds accessor to get information about the problem bounds
            x = np.random.uniform(problem.bounds.lb, problem.bounds.ub)
            self.length = np.linalg.norm(x)
            
            problem(x)  
            

In [12]:
# using the random search algorithm, we can then run a simple experiment
run_experiment(problem, RandomSearch(10))

run: 1 - best found: 126.128
run: 2 - best found: 135.295
run: 3 - best found: 118.337
run: 4 - best found: 154.620
run: 5 - best found: 153.171


# Adding custom problems
As the list of already implemented problems might not contains the objective function you would like to analyze and benchmark, we include an easy to use interface to wrap your benchmark function into an iohexperimenter `problem` object. 

Currently, only single objective functions are supported, so the only requirement on the function is its signature, which should take a single array parameter, and return a single floating point number $f(\mathbf{x}) \mapsto \mathbb{R}$.

In this example we will add the function (Styblinski–Tang):

$f(\mathbf{x}) = \frac{\sum_{i=1}^n x_i^4 - 16x_i^2 + 5x_i}{2}$

defined on $[-5, 5]$

In [13]:
def styblinski_tang(x: np.ndarray) -> float:
    return np.sum(np.power(x, 4) - (16 * np.power(x, 2)) + (5 * x)) / 2


styblinski_tang(np.array([-2.903534]*10)) # global minima

-391.661657037714

In [14]:
# we can wrap this function in ioh, as this is is a continous function, we use wrap_real_problem:
ioh.problem.wrap_real_problem(
    styblinski_tang,                                     # Handle to the function
    name="StyblinskiTang",                               # Name to be used when instantiating
    optimization_type=ioh.OptimizationType.MIN, # Specify that we want to minimize
    lb=-5,                                               # The lower bound
    ub=5,                                                # The upper bound
)

In [15]:
# We can create an instance of this problem, wrapped in ioh
problem = ioh.get_problem("StyblinskiTang", dimension=10)
problem

<RealProblem 25. StyblinskiTang (iid=1 dim=10)>

In [16]:
problem(np.array([-2.903534]*10))

-391.661657037714

## Specifying instances
When doing bechmarking of custom problems, it can often be usefull to look at different instances of the same problem, but that perform some transformation the parameter or objective space, to test if an algorithm is invariant to such transformations. 

Here we add the following transformations to the aforementioned objective function:

$T_x(\mathbf{x}, c) = \mathbf{x} + c$

$T_y(y, c) = y * c$

We can do this by providing transformation functions to the `wrap_problem` interface. Note that these take two parameters:
 1. The operand they function on, which are the variables for the variables transformation and objective function value for the objective transformation. 
 2. An integer identifier of the instance id which is currently used. This allows you to specify alternate behavior for different instances.  

In [17]:
# Variables transformation R^d -> R^d
def transform_variables(x: np.ndarray, instance_id:int) -> np.ndarray:
    c = (instance_id - 1) * .5
    return x + c

# Objective transformation R -> R
def transform_objectives(y: float, instance_id:int) -> float:
    c = instance_id
    return y * c

# Note that we can overwrite a previously defined problem by calling wrap_real_problem again with the same name
ioh.problem.wrap_real_problem(
    styblinski_tang,                                     
    name="StyblinskiTang",                               
    optimization_type=ioh.OptimizationType.MIN, 
    lb=-5,                                               
    ub=5,      
    
    # Adding the transformation functions
    transform_variables=transform_variables,     
    transform_objectives=transform_objectives
)

In [18]:
# We can now create different instances of the same problem
instance1 = ioh.get_problem("StyblinskiTang", instance=1, dimension=10)
instance2 = ioh.get_problem("StyblinskiTang", instance=2, dimension=10)
instance1, instance2

(<RealProblem 25. StyblinskiTang (iid=1 dim=10)>,
 <RealProblem 25. StyblinskiTang (iid=2 dim=10)>)

In [19]:
# Note that when evaluating with the same point, each instance gives a different (transformed value)
instance1(x0), instance2(x0)

(-36.814866821210856, -204.6537242981962)

# Logging data
The default usage of IOHExperimenter is in generating logs of benchmarking experiments which can be analyzed in IOHAnalyzer. 

In [20]:
import os

logger = ioh.logger.Analyzer(
    root=os.getcwd(),                  # Store data in the current working directory
    folder_name="my-experiment",       # in a folder named: 'my-experiment'
    algorithm_name="random-search",    # meta-data for the algorithm used to generate these results
    store_positions=True               # store x-variables in the logged files
)

# this automatically creates a folder 'my-experiment' in the current working directory
# if the folder already exists, it will given an additional number to make the name unique
logger

<Analyzer /home/jacob/code/IOHexperimenter/example/my-experiment>

In [21]:
%ls

CMakeFiles/          experiment_example.h  problem_example.h
CMakeLists.txt       gsemo.cpp             run_submodular_instances.cpp
README.md            logger_example.h      suite_example.h
add_new_problem.cpp  main.cpp              tutorial.ipynb
eafh.cpp             my-experiment/


In [22]:
# In order to log data for a problem, we only have to attach it to a logger
problem = ioh.get_problem("StyblinskiTang", instance=1, dimension=2)
problem.attach_logger(logger)

# We can then run the random search as before, only now all data will be logged to a file
run_experiment(problem, RandomSearch(10), n_runs=1)

run: 1 - best found:-62.781


In [23]:
logger.close()

In [24]:
%cat my-experiment/IOHprofiler_f25_StyblinskiTang.json

{
	"version": "0.3.3", 
	"suite": "unknown_suite", 
	"function_id": 25, 
	"function_name": "StyblinskiTang", 
	"maximization": false, 
	"algorithm": {"name": "random-search", "info": "algorithm_info"},
	"attributes": ["evaluations", "raw_y"],
	"scenarios": [
		{"dimension": 2,
		"path": "data_f25_StyblinskiTang/IOHprofiler_f25_DIM2.dat",
		"runs": [
			{"instance": 1, "evals": 10, "best": {"evals": 6, "y": -62.780970416619525, "x": [-2.7481191615752754, 2.4684405799397533]}}
		]}
	]
}


In [25]:
%cat my-experiment/data_f25_StyblinskiTang/IOHprofiler_f25_DIM2.dat

evaluations raw_y x0 x1
1 -62.3085384524 -1.697436 -2.909544
6 -62.7809704166 -2.748119 2.468441
10 -11.3065941964 0.728007 -0.939917


## Triggers 
The default behavior of the `Analyzer` logger is to log data only when there is an improvement of the objective value. We can change this behaviour, by specifying one or more triggers, which are logical operators, which when one of them evaluates to True, will cause data to be logged.

We provide a number of trigger variants which can be use to customize the logging. In the following example, a trigger is defined which evaluates to True, every 3 function evaluations. It is combined with a trigger for improvement, so data will be logged on every 3rd function evaluation, or when there is an observed improvement of the objective value

In [26]:
triggers = [
    ioh.logger.trigger.Each(3),
    ioh.logger.trigger.OnImprovement()
]

logger = ioh.logger.Analyzer(
    root=os.getcwd(),                  
    folder_name="my-experiment",       
    algorithm_name="random-search",    
    store_positions=True,
    
    # Add the triggers to the logger
    triggers = triggers
)

logger

<Analyzer /home/jacob/code/IOHexperimenter/example/my-experiment-1>

In [27]:
# Rerun the same experiment as before
problem = ioh.get_problem("StyblinskiTang", instance=1, dimension=2)
problem.attach_logger(logger)
run_experiment(problem, RandomSearch(10), n_runs=1)

run: 1 - best found:-61.287


In [28]:
# We can now see that data is logged either if there is improvement, or on every 3rd evaluation
%cat my-experiment-1/data_f25_StyblinskiTang/IOHprofiler_f25_DIM2.dat

evaluations raw_y x0 x1
1 -25.5404960631 -0.719737 -1.525905
2 -33.2194759361 -4.082862 -1.978192
3 85.1980583764 4.208977 -4.730113
5 -61.2873120868 -2.587149 3.035593
6 -29.7107630740 -0.649874 -1.780952
9 -60.2564799291 -1.842026 -2.312471
10 57.9991264381 4.530844 0.183927


## Properties
If we want to keep track of any dynamic parameters a given algorithm might have, we can use properties to log them to the output files. 

In the following example, we will track the length parameters for the RandomSearch algorithm, which is added for illustrative purpoposes, and changes for every function evaluation

In [29]:
# RandomSearch has a length parameter, which is dynamic
algorithm = RandomSearch(10)

# Creating a new logger
logger = ioh.logger.Analyzer(
    root=os.getcwd(),                  
    folder_name="my-experiment",       
    algorithm_name="random-search",    
    store_positions=True
)

# Before we attach a problem, we tell the logger to keep track of the length parameter on algorithm
logger.watch(algorithm, "length")

# We can now again run the same experiment 
problem = ioh.get_problem("StyblinskiTang", instance=1, dimension=2)

problem.attach_logger(logger)
run_experiment(problem, algorithm, n_runs=1)

run: 1 - best found:-37.976


In [38]:
logger.close() 

In [31]:
# Note the additional length parameter being logged 
%cat my-experiment-2/data_f25_StyblinskiTang/IOHprofiler_f25_DIM2.dat

evaluations raw_y length x0 x1
1 -37.9757402234 2.5096422773 0.768772 -2.388994
10 -8.3519594896 3.6807063542 3.676021 -0.185660


# Alternate logging behaviour
We provide a number of different loggers in addition to the `Analyzer` logger, which include:
 - `FlatFile` which logs data to a simple csv file
 - `Store` which keeps all of the stored data in memory
 - `EAF/EAH` which compute Empirical Attainment Function/Histogram statistics on the fly

You can define your own custom logging behavoir by inheriting from the `AbstractLogger` class. The only required part of the interface is that you override the `__call__` operator, which takes a single `ioh.LogInfo` parameter. In this method you should define your desired behavior. 

In [32]:
# Simple logger that logs data using the python logging module whenever it is triggeredd
class MyLogger(ioh.logger.AbstractLogger):
    def __call__(self, log_info: ioh.LogInfo):
        logging.info(msg=f"triggered! y: {log_info.y}")


# The abstract logger takes two parameters, triggers and properties
mylogger = MyLogger(triggers=[ioh.logger.trigger.ALWAYS])

problem = ioh.get_problem("StyblinskiTang", instance=1, dimension=2)
problem.attach_logger(mylogger)

run_experiment(problem, RandomSearch(10), 1)

01:51:42 INFO - triggered! y: -42.48236735512947
01:51:42 INFO - triggered! y: -18.44669950795916
01:51:42 INFO - triggered! y: 2.00311342984765
01:51:42 INFO - triggered! y: 90.08208841172612
01:51:42 INFO - triggered! y: 24.49818702405043
01:51:42 INFO - triggered! y: -30.4413351456828
01:51:42 INFO - triggered! y: -0.6379516707627677
01:51:42 INFO - triggered! y: -5.684436871723673
01:51:42 INFO - triggered! y: 96.83328267284668
01:51:42 INFO - triggered! y: -46.440671419109655


run: 1 - best found:-46.441


# Standardized Experimental Setup (Python only)
In Python, we provide the `Experiment` class which can be used to easily run a given algorithm over a larger number of problems. 

In [33]:
experiment = ioh.Experiment(
    algorithm = RandomSearch(10), # An algorithm instance
    fids = [1, 25],               # the id's of the problems we want to test
    iids = [1, 10],               # the instances 
    dims = [2, 10],               # the dimensions
    reps = 3,                     # the number of runs,
    zip_output = True       
)

In [34]:
experiment.run()

In [35]:
%ls ioh_data

IOHprofiler_f1_Sphere.json           data_f1_Sphere/
IOHprofiler_f25_StyblinskiTang.json  data_f25_StyblinskiTang/


In [37]:
%cat ioh_data/IOHprofiler_f25_StyblinskiTang.json

{
    "version": "0.3.3",
    "suite": "unknown_suite",
    "function_id": 25,
    "function_name": "StyblinskiTang",
    "maximization": false,
    "algorithm": {
        "name": "RandomSearch(n=10, length=0.0)",
        "info": "algorithm_info"
    },
    "attributes": [
        "evaluations",
        "raw_y"
    ],
    "scenarios": [
        {
            "dimension": 10,
            "path": "data_f25_StyblinskiTang/IOHprofiler_f25_DIM10.dat",
            "runs": [
                {
                    "instance": 10,
                    "evals": 10,
                    "best": {
                        "evals": 6,
                        "y": 956.1600512285518,
                        "x": [
                            -4.115877772687728,
                            -4.862972597469572,
                            -3.7296284770815524,
                            -3.3497968315431947,
                            -2.5477172826748617,
                            -3.4376672121575416,
   

In [1]:
%ls 

CMakeFiles/           ioh_data/         my-experiment-2/
CMakeLists.txt        ioh_data.zip      problem_example.h
README.md             logger_example.h  run_submodular_instances.cpp
add_new_problem.cpp   main.cpp          suite_example.h
eafh.cpp              my-experiment/    tutorial.ipynb
experiment_example.h  my-experiment-1/  tutorial.py


In [ ]:
# cleanup
%rm -rf my-experiment*
%rm -rf ioh_data*